# 한글 텍스트 처리 - 네이버 영화 평점 감성 분석

In [1]:
import pandas as pd

train_df = pd.read_csv('./data/ratings_train.txt', sep = '\t', encoding = 'utf-8')

train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [2]:
train_df['label'].value_counts()

label
0    75173
1    74827
Name: count, dtype: int64

문자가 아닌 숫자의 경우 단어적인 의미로 부족하므로 파이썬의 정규 표현식 모듈인 re를 이용해 공백으로 변환

In [3]:
import re

train_df = train_df.fillna(' ')

# 정규 표현식을 이용해 숫자를 공백으로 변경(정규 표현식으로 \d는 숫자를 의미)
train_df['document'] = train_df['document'].apply(lambda x : re.sub(r"\d+", " ", x))

# 체스트 데이터 세트를 로딩하고 동일하게 null 및 숫자를 공백으로 변환
test_df = pd.read_csv('./data/ratings_test.txt', sep = '\t', encoding = 'utf-8')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x : re.sub(r"\d+", " ", x))

In [4]:
# id 컬럼 삭제

triain_df = train_df.drop('id', axis = 1)
test_df = test_df.drop('id', axis = 1)

In [8]:
# 입력 인자로 들어온 문장을 형태소 단어 형태로 토큰화해 list 객체로 반환

from konlpy.tag import Twitter
import warnings
warnings.filterwarnings('ignore')

twitter = Twitter()
def tw_tokenizer(text):
    # 입력 인자로 들어온 텍스틀 형태소 단어로 토크화해 리스트 형태로 반환
    tokens_ko = twitter.morphs(text)
    return tokens_ko

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Twitter 객체의 morphs() 객체를 이용한 tokenizer를 사용
tfidf_vect = TfidfVectorizer(tokenizer = tw_tokenizer, ngram_range = (1, 2), min_df = 3, max_df = 0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

In [12]:
# 로지스틱 회귀를 이용해 감성 분석 분류 수행
lg_clf = LogisticRegression(random_state = 0, solver = 'liblinear')

# 파라미터 C 최적화를 위해 GridSerachCV를 이용
params = {'C' : [1, 3.5, 4.5, 5.5, 10]}
grid_cv = GridSearchCV(lg_clf, param_grid = params, cv = 3, scoring = 'accuracy', verbose = 1)
grid_cv.fit(tfidf_matrix_train, train_df['label'])
print(grid_cv.best_params_, round(grid_cv.best_score_, 4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 3.5} 0.8593


In [13]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용해 테스트 데이터를 TF-IDF 값으로 피처 변환
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 사용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도 : ', accuracy_score(test_df['label'], preds))

Logistic Regression 정확도 :  0.86172
